In [0]:
%sql
SELECT anomes,
       ROUND(AVG(total_amount), 2) AS media_total_amount
FROM workspace.nyc_taxi.yellowtaxi_trips_2023_silver
GROUP BY anomes
ORDER BY anomes;


In [0]:
%sql
SELECT HOUR(tpep_pickup_datetime) AS hora_do_dia,
       ROUND(AVG(passenger_count), 2) AS media_passageiros
FROM workspace.nyc_taxi.yellowtaxi_trips_2023_silver
WHERE anomes = '202305'
GROUP BY hora_do_dia
ORDER BY hora_do_dia;


In [0]:
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

# 1. Consulta e coleta os dados
df_maio = (spark.table("workspace.nyc_taxi.yellowtaxi_trips_2023_silver")
    .filter(F.col("anomes") == "202305")
    .groupBy(F.hour("tpep_pickup_datetime").alias("hora_do_dia"))
    .agg(F.round(F.avg("passenger_count"), 2).alias("media_passageiros"))
    .orderBy("hora_do_dia")
)

# Converte para pandas para plotar
pdf = df_maio.toPandas()

# 2. Cria o gráfico
plt.figure(figsize=(10,5))
plt.plot(pdf["hora_do_dia"], pdf["media_passageiros"], marker="o")
plt.title("Média de Passageiros por Hora - Maio/2023")
plt.xlabel("Hora do Dia")
plt.ylabel("Média de Passageiros")
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()
